In [1]:
%load_ext autoreload
%autoreload 2

In [6]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
from torch.utils.data import Dataset

In [3]:
device = "cuda" if torch.cuda.is_available() else "cpu"
device

'cuda'

In [4]:
from utils import load_csv, load_images, fill_na, plot_landmark, plot_landmark_single

In [8]:
class FaceLandmarkDataset(Dataset):
    def __init__(self, device='cuda', transform=None):
        self.device = device
        self.landmarks_frame = load_csv()
        fill_na(self.landmarks_frame)
        images = load_images()
        self.images =torch.from_numpy(images).type(torch.float)
        
        self.transform = transform
    
    def __len__(self):
        return len(self.landmarks_frame)
    
    def __getitem__(self, idx):
        if torch.is_tensor(idx):
            idx = idx.tolist()
        
        img = self.images[idx]
        landmarks = torch.from_numpy(self.landmarks_frame.iloc[idx].to_numpy()).type(torch.float64)
        
        sample = sample = {'image': img, 'landmarks': landmarks}
        
        if self.transform:
            sample = self.transform(img, landmarks)
        
        
        return sample

In [9]:
ds = FaceLandmarkDataset()

/home/jupyteruser/utils.py:16: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df[i].fillna(df[i].mean(),inplace=True)
/home/jupyteruser/utils.py:8: UserWarning: Reading `.npy` or `.npz` file required additional header parsing as it was created on Python 2. Save the file again to speed up loading and avoid this warning.
  images = np.moveaxis(data_np['face_images'], -1,0)


In [11]:
item = ds.__getitem__(0)

In [12]:
item['landmarks']

tensor([66.0336, 39.0023, 30.2270, 36.4217, 59.5821, 39.6474, 73.1303, 39.9700,
        36.3566, 37.3894, 23.4529, 37.3894, 56.9533, 29.0336, 80.2271, 32.2281,
        40.2276, 29.0023, 16.3564, 29.6475, 44.4206, 57.0668, 61.1953, 79.9702,
        28.6145, 77.3890, 43.3126, 72.9355, 43.1307, 84.4858],
       dtype=torch.float64)

In [16]:
def pairwise(iterable):
    a = iter(iterable)
    return zip(a,a)

In [18]:
p = []
p_x = []

for (x,y) in pairwise(item['landmarks'].numpy()):
    p.append((x,y))
    p_x.append(x)
    p_x.append(y)

print(p)
print(p_x)

[(66.0335639098, 39.0022736842), (30.2270075188, 36.4216781955), (59.582075188, 39.6474225564), (73.1303458647, 39.9699969925), (36.3565714286, 37.3894015038), (23.4528721805, 37.3894015038), (56.9532631579, 29.0336481203), (80.2271278195, 32.2281383459), (40.2276090226, 29.0023218045), (16.3563789474, 29.6474706767), (44.4205714286, 57.0668030075), (61.1953082707, 79.9701654135), (28.6144962406, 77.3889924812), (43.3126015038, 72.9354586466), (43.1307067669, 84.4857744361)]
[66.0335639098, 39.0022736842, 30.2270075188, 36.4216781955, 59.582075188, 39.6474225564, 73.1303458647, 39.9699969925, 36.3565714286, 37.3894015038, 23.4528721805, 37.3894015038, 56.9532631579, 29.0336481203, 80.2271278195, 32.2281383459, 40.2276090226, 29.0023218045, 16.3563789474, 29.6474706767, 44.4205714286, 57.0668030075, 61.1953082707, 79.9701654135, 28.6144962406, 77.3889924812, 43.3126015038, 72.9354586466, 43.1307067669, 84.4857744361]


In [19]:
import random
import math
import torchvision.transforms.functional as F
from PIL import Image

class RandomRotate(object):
    def __init__(self, degrees):
        self.degrees = degrees
    
    def __call__(self, image, points):
        angle = random.uniform(-self.degrees, self.degrees)
        
        image = F.rotate(image, points, resample=Image.BILNEAR)
        
        radians = math.radians(angle)
        cos_theta = math.cos(radians)
        sin_theta = math.sin(radians)
        rotated_points = []
        
        for (x,y) in points:
            x_rot = x * cos_theta - y * sin_theta
            y_rot = x * sin_theta + y * cos_theta
            rotated_points.append(x_rot)
            rotated_points.append(y_rot)
        
        return {'image': image,  'landmarks': torch.tensor(rotated_points)}

In [20]:
class FacialFeaturesV6(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Conv2d(1, 512, 2)
        self.conv2 = nn.Conv2d(512, 256, 2)
        self.conv3 = nn.Conv2d(256, 128, 2)
        self.pool = nn.MaxPool2d(2,2)
        self.conv4 = nn.Conv2d(128, 128, 3)
        self.conv5 = nn.Conv2d(128, 64, 3)
        self.conv6 = nn.Conv2d(64, 32, 3)
        self.batch_norm_l = nn.BatchNorm2d(128)        
        self.batch_norm_m = nn.BatchNorm2d(64)        
        self.batch_norm = nn.BatchNorm2d(32)        
        self.fc1 = nn.Linear(32, 128)
        self.fc2 = nn.Linear(128, 256)
        self.fc3 = nn.Linear(256, 128)
        self.fc4 = nn.Linear(128, 64)
        self.out_layer = nn.Linear(64, 30)
        self.relu = nn.ReLU()
        
    def forward(self, x):
        x = self.relu(self.conv1(x))
        x = self.relu(self.conv2(x))
        x = self.pool(x)
        x = self.relu(self.conv3(x))
        x = self.batch_norm_l(x)
        x = self.pool(x)
        x = self.relu(self.conv4(x))
        x = self.pool(x)
        x = self.relu(self.conv5(x))
        x = self.pool(x)
        x = self.relu(self.conv6(x))
        x = self.batch_norm(x)
        x = self.pool(x)
        x = torch.flatten(x, 1)
        x = self.relu(self.fc1(x))
        x = self.relu(self.fc2(x))
        x = self.relu(self.fc3(x))
        x = self.relu(self.fc4(x))
        x = self.relu(self.out_layer(x))
        
        return x